## Exercise 2


In [1]:
import math

In [145]:
def RiceEncode(value, k):
    m = math.pow(2, k)
    
    quotient =  int(math.floor(value/m))
    remainder = int(value % m)
    
    quotientCode = ('1' * quotient) + '0'
    remainderCode = format(remainder, f'0{k}b')
    
    encodedValue = quotientCode + remainderCode
    
    print(f'value: {value}, {bin(value)[2:]}, k: {k}')
    print(f'quotient: {quotient}, remainder: {remainder}')
    print(f'quotient_code: {quotientCode}, remainder_code: {remainderCode}')
    print(f'encoded_value: {encodedValue}')
    return int(encodedValue, 2)

def RiceDecode(value, k):
    m = math.pow(2, k)
    strValue = bin(value)[2:]
    indexOfFirstZero = str(strValue).index('0')
    print(f'index of first zero: {indexOfFirstZero}')
    
    quotientCode = strValue[0:indexOfFirstZero]
    remainderCode = strValue[indexOfFirstZero + 1:]
    print(f'quotient code: {quotientCode}0, remainder code: {remainderCode}')
    
    quotient = len(quotientCode)
    remainder = int(remainderCode, 2)
    print(f'quotient: {quotient}, remainder: {remainder}')
    
    decodedValue = int(quotient * m + remainder)
    return decodedValue


def EncodeFileWithRiceEncode(source, destination, k, bufSize):
    #Read from source byte by byte.
    #Encode each byte. The encoded value can be longer or shorter than a byte.
    #If it is longer, split it in chunks one-byte long and add it to the buffer
    #If it is shorter, packs it into one byte and fill the remaining bits with the next chunk
    
    #buffer
    buffer = bytearray()
    #temporary byte where to store bits
    dByte = 0
    #bits left empty in the dByte
    bitsLeft = 8
    
    with open(source, "rb") as sStream:
        with open(destination, "wb") as dStream:
            #read from source byte-by-byte
            while (sByte := sStream.read(1)):
                #print(f'sByte: {int.from_bytes(sByte, "big")}')
                #Encode the byte
                iEncByte = RiceEncode(int.from_bytes(sByte, "big"), k);
                sEncByte = bin(iEncByte)[2:]
                #print(f'sByte: { int.from_bytes(sByte, "big")}, iEncByte: { iEncByte}, sEncByte: { sEncByte}')
                #index store the current position in the encoded value
                index = 0
                #pack the encode value in chunks into dByte
                while(index < len(sEncByte)):
                    #read bytes from index to nextIndex, max 8 bits, not going beyong the end of encoded value
                    nextIndex = min(index+bitsLeft, len(sEncByte))
                    #how many bits left empty in dByte
                    bitsLeft -= (nextIndex - index)
                    #pack the bits into dByte
                    #If the number of bits to pack fills dByte, there is no bit shift to do
                    #If the number of bits to pack is shorter than the number of bits left, 
                    #shift the bits to the left so as to pack them into the dByte and left the free space on the right side
                    dByte |= (int(sEncByte[index:nextIndex], 2) << bitsLeft)
                    
                    #If there are no bits left, meaning that the dByte is fully packed
                    if(bitsLeft == 0):
                        #add the dByte to the buffer
                        buffer.append(dByte)
                        #empty the dByte for next iteration
                        dByte = 0
                        #reset the number of available bits in dByte
                        bitsLeft = 8
                        #Check the buffer, if its full (its size is greater than or equal to the buffers size set in bufSize)
                        if(len(buffer) >= bufSize):
                            #write the buffer into the destination stream
                            dStream.write(buffer)
                            #empty the buffer
                            buffer = bytearray()
                    
                    #Update the index for the next iteration throught the encoded value
                    index = nextIndex
                
            #If sStream is EOF but there is a dByte partially filled, add it to the buffer and flush
            if(bitsLeft != 8):
                buffer.append(dByte)
                
            dStream.write(buffer)

def GetFirstIndexOfZero(byte):
    for i in range(0, 8):
        mask = 1 << (7 - i)
        if (~byte) & mask != 0:
            return i
    return -1

def GetRightExtractionMask(numOfBits):
    mask = 0
    for i in range(0, numOfBits):
        mask |= (1 << i)
    return mask

def ZeroizeBitsFromLeftMask(numOfBits):
    mask = 0
    for i in range(0, numOfBits):
        mask |=(1 << (7 - i))
    return ~mask
            
def DecodeFileWithRiceEncode(source, destination, k, bufSize):

    buffer = bytearray()
    index = 0
    with open(source, "rb") as sStream:
        with open(destination, "wb") as dStream:
            while (sByte := sStream.read(1)): 
                #print(f'byte read {sByte[0]}')
                indexOfZero = GetFirstIndexOfZero(sByte[0])
                
                if indexOfZero == -1:
                    buffer.append(sByte[0])
                    #print(f'Added to buffer 1 {sByte[0]}')
                    continue
                
                if indexOfZero + k > 8:
                    buffer.append(sByte[0])
                    #print(f'Added to buffer 2 {sByte[0]}')
                    
                    #read one more byte
                    sByte= sStream.read(1)
                    #print(f'byte read {sByte[0]}')
                
                    shift = (8 + 7 - (indexOfZero + k))
                else:
                    shift = 7 - (indexOfZero + k)
                
                #print(f'index of zero: {indexOfZero}, shift: {shift}')
                sByteAsInt = int.from_bytes(sByte, "big")
                
                #clean leftmost 8-shift bits
                zeroizeMask = ZeroizeBitsFromLeftMask(8-shift)
                remByte = sByteAsInt & zeroizeMask
                #print(f'remaining {remByte}')
                #add remByte to the next iteration's buffer

                #add current byte to bytearray
                #note that the rigthmost portion of the current byte (which belongs to the next value)
                #will be discarded in the next for loop
                
                buffer.append(sByteAsInt)
                #print(f'Added to buffer 3 {sByteAsInt}')
                    
                for i in range(len(buffer)-1, -1, -1):

                    if(i < len(buffer)-1):

                        #move rigthmost shift bits all the way to the left side
                         
                        extr = buffer[i] & zeroizeMask
                        extr = extr << (8 - shift)
                        
                        #print(f'buffer[i] {buffer[i]}, extr: {extr}, type: {type(extr)}')
                        #move that to the previous byte
                        buffer[i+1] |= extr


                    #shift left by rShift
                    #print(f'buffer[i] {buffer[i]}')
                    buffer[i] >>= shift
                    #print(f'after shift: {buffer[i]}')

                #combine bytearray in one number


                valueToDecipher = int.from_bytes(buffer, "big")
                #decifer byte
                decypheredValue = RiceDecode(valueToDecipher, k)
                
                #print(f'valueToDecipher: {valueToDecipher}; decypheredValue: {decypheredValue}')
                
                
                #write value to dest stream
                dStream.write(int.to_bytes(decypheredValue, 1, "big"))
                #clean bytearray
                buffer = bytearray()
                #add leftover byte to buffer
                buffer.append(remByte)
                        
                    
                #print(f'sByte: {sByte}, index:{indexOfZero}')
                #break
                

In [146]:
originalFile = "Exercise2_Files\Sound1.wav"
encodedFile = "Exercise2_Files\Sound1_Enc.ex2"
decodedFile = "Exercise2_Files\Sound1_Enc_Dec.wav"

k = 4
bufSize = 256

#EncodeFileWithRiceEncode(originalFile, encodedFile, k, bufSize)

DecodeFileWithRiceEncode(encodedFile, decodedFile, k, bufSize)

print("Done!")


index of first zero: 5
quotient code: 111110, remainder code: 0010
quotient: 5, remainder: 2
index of first zero: 4
quotient code: 11110, remainder code: 100111
quotient: 4, remainder: 39
index of first zero: 2
quotient code: 110, remainder code: 01101
quotient: 2, remainder: 13
index of first zero: 3
quotient code: 1110, remainder code: 0110
quotient: 3, remainder: 6
index of first zero: 6
quotient code: 1111110, remainder code: 0000
quotient: 6, remainder: 0


ValueError: negative shift count

In [86]:
buf = bytearray()
#buf.append(15)
#buf.append(10)
#buf.append(10)
#val = int.from_bytes(buf, "big")
#print(val)

buf.append(248)
#val = buf[0] << 2

print (buf[0])

val = buf[0]>> 4

print(val)



248
15


In [144]:
k = 4
value = 82

encodedValue = RiceEncode(int(value), k)
print(f'value: {value}, encoded value: {encodedValue}')

decodedValue = RiceDecode(encodedValue, k)
print(f'decoded value: { decodedValue}')

value: 82, 1010010, k: 4
quotient: 5, remainder: 2
quotient_code: 111110, remainder_code: 0010
encoded_value: 1111100010
value: 82, encoded value: 994
index of first zero: 5
quotient code: 111110, remainder code: 0010
quotient: 5, remainder: 2
decoded value: 82
